In [3]:
from PIL import Image
from PIL.TiffTags import TAGS
import re
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.transform import Affine
import numpy as np 
import cv2
import imutils
import matplotlib.pyplot as plt



In [8]:
def metadata(imagen):
    img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
    meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}
    # Se imprime el diccionario para obtener la composición de los datos
    # for rec in meta_dict:
    #     print(rec, ":", meta_dict[rec])

    # Extraemos el indicador xmp
    p = meta_dict.get("XMP")
    s = p.decode("UTF-8")
    # dividimos por el salto de linea y obtenemos una lista
    div = s.split("\n")

    #eliminamos los espacios vacios 

    for ind,recorrido in enumerate(div):
        div[ind]=recorrido.strip() 
    usar = div[17]

    result = re.search(":(.*)=", div[17])
    result.group(1)

    result2 = re.search("\"(.*)\"", div[17])
    result2.group(1)

    metadiccionario = {}
    for ind,recorrido in enumerate(div):
        try:
            metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
        except:
            pass

    metadiccionario = {}
    for ind,recorrido in enumerate(div):
        try:
            metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
        except:
            pass

    #print(metadiccionario)
    return metadiccionario

def TransfromRaster(img_path, xy_in):
    metadiccionario = metadata(img_path)
    
    altura_vuelo = float(metadiccionario["RelativeAltitude"])
    distancia_focal = float(metadiccionario["CalibratedFocalLength"])

    resolucion = altura_vuelo/distancia_focal
    print(metadiccionario["GpsLongitude"])
    print(metadiccionario["GpsLatitude"])
    min_lon = (float(metadiccionario["GpsLongitude"])) - (
        float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
    max_lon = (float(metadiccionario["GpsLongitude"])) + (
        float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
    min_lat = (float(metadiccionario["GpsLatitude"])) - (
        float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
    max_lat = (float(metadiccionario["GpsLatitude"])) + (
        float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111

    img_data = rasterio.open(img_path, 'r')

    bands = [1] #Se especifica las cantidades de canales que tiene la imagen.

    data = img_data.read(bands)
    _, width, height = data.shape #width corresponde al eje x y height al eje y

    transform = rasterio.transform.from_bounds(
            min_lon, min_lat, max_lon, max_lat, height, width)
    
    transform_xy_to_latlon = np.array([[transform.a, transform.b, transform.c], 
                     [transform.d, transform.e, transform.f],
                     [0, 0, 1]], dtype =np.float64)

    
    xy = np.array([[xy_in[0]],
                   [xy_in[1]],
                   [1]], dtype=np.float64)
    
    print(np.dot(transform_xy_to_latlon, xy))


In [11]:
img_path = "Img/DJI_0011.TIF"
img_ruta = "Img/ll.TIF"

xy_in = [850, 100]

trans = TransfromRaster(img_path, xy_in)


-74.18553757
11.22467042
[[-74.18553171]
 [ 11.22473484]
 [  1.        ]]


In [43]:
with rasterio.open(        
        img_ruta,  # Nombre del nuevo archivo georefenciado
        'r',
        driver='GTiff',
        ) as dts:
            print(dts.xy(650, 800))

(-74.18553751143722, 11.224670361437225)


In [46]:
transfor = np.array([[trans.a, trans.b, trans.c], 
                     [trans.d, trans.e, trans.f],
                     [0, 0, 1]], dtype =np.float64)

xy = np.array([[650],
               [800],
               [1]], dtype=np.float64)

np.dot(transfor, xy)

array([[-74.18555514],
       [ 11.22465285],
       [  1.        ]])